In [10]:
import pandas as pd
from datetime import datetime, timedelta
from preprocess_fn import id_encode, ignore_first, only_keep_last
from file_reader import FileReader

# 전처리 내용
* 각 데이터 열 이름 매칭
* 각 데이터별 상위 카테고리 추가 -> {'Residential', 'Commercial/Services', 'Educational', 'Transportation', 'Culture & Leisure', 'Healthcare & Welfare'}
* 각 데이터별 요일 정보 추가 -> 쉬는 날만 구분
* 각 check-in 시간대 조정 -> 하루중 상대시간으로 설정
* 각 check-in 단위 trajectory 생성

# NYC 데이터

In [11]:
import pandas as pd

nyc_train = pd.read_csv('../data/nyc/raw_basic_ver/NYC_train.csv')
nyc_val = pd.read_csv('../data/nyc/raw_basic_ver/NYC_val.csv')
nyc_test = pd.read_csv('../data/nyc/raw_basic_ver/NYC_test.csv')

nyc_train

,user_id,POI_id,POI_catid,POI_catid_code,POI_catname,latitude,longitude,timezone,UTC_time,local_time,day_of_week,norm_in_day_time,trajectory_id,norm_day_shift,norm_relative_time
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,75,Arts & Crafts Store,40.719810,-74.002581,-240,2012-04-03 18:00:09+00:00,2012-04-03 14:00:09+00:00,1,0.583333,470_1,0,0.583333
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,252,Bridge,40.606800,-74.044170,-240,2012-04-03 18:00:25+00:00,2012-04-03 14:00:25+00:00,1,0.583333,979_1,0,0.583333
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,6,Home (private),40.716162,-73.883070,-240,2012-04-03 18:02:24+00:00,2012-04-03 14:02:24+00:00,1,0.583333,69_1,0,0.583333
3,428,4ce1863bc4f6a35d8bd2db6c,4bf58dd8d48988d103941735,6,Home (private),40.619151,-74.035888,-240,2012-04-03 18:06:18+00:00,2012-04-03 14:06:18+00:00,1,0.583333,428_1,0,0.583333
4,116,4c0ab56f7e3fc9288c1df482,4f04afc02fb6e1c99f3db0bc,362,Mobile Phone Shop,40.741191,-73.989663,-240,2012-04-03 18:09:29+00:00,2012-04-03 14:09:29+00:00,1,0.583333,116_1,0,0.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83223,529,50170a53e4b07ac9f310b944,4bf58dd8d48988d1e0931735,253,Coffee Shop,40.709272,-74.005450,-300,2012-11-09 20:45:40+00:00,2012-11-09 15:45:40+00:00,4,0.645833,529_56,0,0.645833
83224,354,4a9fc4fbf964a520493d20e3,4bf58dd8d48988d129951735,79,Train Station,40.747582,-74.171888,-300,2012-11-09 20:57:51+00:00,2012-11-09 15:57:51+00:00,4,0.645833,354_39,0,0.645833
83225,472,4d056fc57d9ba35d1c446b23,4bf58dd8d48988d130941735,91,Building,40.760184,-73.969324,-300,2012-11-09 21:04:32+00:00,2012-11-09 16:04:32+00:00,4,0.666667,472_40,0,0.666667
83226,280,4ae1f681f964a520318921e3,4bf58dd8d48988d103951735,7,Clothing Store,40.753716,-73.980947,-300,2012-11-09 21:11:40+00:00,2012-11-09 16:11:40+00:00,4,0.666667,280_51,0,0.666667


In [17]:
nyc_train.loc[nyc_train['user_id']==1]

,user_id,POI_id,POI_catid,POI_catid_code,POI_catname,latitude,longitude,timezone,UTC_time,local_time,day_of_week,norm_in_day_time,trajectory_id,norm_day_shift,norm_relative_time
7382,1,46ea2358f964a520cf4a1fe3,4bf58dd8d48988d11d941735,56,Bar,40.760667,-73.994948,-240,2012-04-14 01:11:20+00:00,2012-04-13 21:11:20+00:00,4,0.875000,1_4,0,0.875000
7571,1,40fb0f00f964a520d90a1fe3,4bf58dd8d48988d11b941735,53,Bar,40.760645,-73.986065,-240,2012-04-14 04:45:13+00:00,2012-04-14 00:45:13+00:00,5,0.020833,1_4,1,1.020833
7917,1,428d2880f964a520b5231fe3,4bf58dd8d48988d1fa931735,301,Hotel,40.756731,-73.974070,-240,2012-04-14 17:45:23+00:00,2012-04-14 13:45:23+00:00,5,0.562500,1_4,1,1.562500
8221,1,4e742aabc65bb91db3cadb79,4bf58dd8d48988d116941735,43,Bar,40.757305,-73.968640,-240,2012-04-14 22:07:43+00:00,2012-04-14 18:07:43+00:00,5,0.750000,1_4,1,1.750000
8309,1,3fd66200f964a520fbe71ee3,4bf58dd8d48988d16c941735,148,Burger Joint,40.758819,-73.968657,-240,2012-04-14 23:15:07+00:00,2012-04-14 19:15:07+00:00,5,0.791667,1_4,1,1.791667
12800,1,49d2b43ef964a520cb5b1fe3,4bf58dd8d48988d1e0931735,253,Coffee Shop,40.720087,-74.003961,-240,2012-04-19 19:02:25+00:00,2012-04-19 15:02:25+00:00,3,0.625000,1_5,0,0.625000
13021,1,4aab0204f964a520855820e3,4bf58dd8d48988d14e941735,118,American Restaurant,40.741319,-74.006116,-240,2012-04-19 22:51:19+00:00,2012-04-19 18:51:19+00:00,3,0.770833,1_5,0,0.770833
13238,1,3fd66200f964a520e9e61ee3,4bf58dd8d48988d147941735,111,Diner,40.733763,-74.006264,-240,2012-04-20 01:49:26+00:00,2012-04-19 21:49:26+00:00,3,0.895833,1_5,0,0.895833
14313,1,467ffefcf964a52019481fe3,4bf58dd8d48988d10c941735,24,French Restaurant,40.779180,-73.950852,-240,2012-04-21 00:15:18+00:00,2012-04-20 20:15:18+00:00,4,0.833333,1_5,1,1.833333
14943,1,4e691e82b9930387355c629d,4bf58dd8d48988d14e941735,118,American Restaurant,40.725163,-73.992160,-240,2012-04-21 16:28:11+00:00,2012-04-21 12:28:11+00:00,5,0.500000,1_5,2,2.500000


* NYC 추가 전처리

In [10]:
# 1. 열이름 매칭
def make_same_column_names(df):
    df.rename(columns={
        'user_id': 'UserId',
        'POI_id': 'PoiId',
        'POI_catid': 'PoiCategoryId',
        'POI_catname': 'PoiCategoryName',
        'latitude': 'Latitude',
        'longitude': 'Longitude',
        'timezone': 'TimezoneOffset',
        'local_time': 'LocalTime',
        'day_of_week': 'DayOfWeek',
        'norm_in_day_time': 'NormInDayTime',
    }, inplace=True)
    return df

# 2. 공휴일 정보 추가 및 시간대 확인
def add_holidays_nyc(df):
    df['LocalTime'] = pd.to_datetime(df['LocalTime'])

    # 미국 공휴일 정의 (2012년 4월~2013년 2월)
    us_holidays = [
        datetime(2012, 4, 6).date(),  # Good Friday
        datetime(2012, 5, 28).date(), # Memorial Day
        datetime(2012, 7, 4).date(),  # Independence Day
        datetime(2012, 9, 3).date(),  # Labor Day
        datetime(2012, 11, 22).date(),# Thanksgiving Day
        datetime(2012, 12, 25).date(),# Christmas Day
        datetime(2013, 1, 1).date(),  # New Year's Day
        datetime(2013, 1, 21).date()  # Martin Luther King Jr. Day
    ]
    
    # 공휴일 정보 추가
    df['is_weekend'] = df['LocalTime'].dt.weekday.isin([5, 6])  # 토요일(5), 일요일(6)
    df['is_holiday'] = df['LocalTime'].isin(us_holidays)

    df['Holiday'] = df['is_weekend'] | df['is_holiday']
    df.drop(columns=['is_weekend', 'is_holiday'], inplace=True)
    
    return df

# 3. 상위 카테고리 맵핑
final_category_mapping = {
    'Residential': [
        'Home (private)', 'Neighborhood', 'Residential Building (Apartment / Condo)', 
        'Housing Development', 'Hotel'
    ],
    'Commercial/Services': [
        'Food & Drink Shop', 'Burger Joint', 'Coffee Shop', 'Ice Cream Shop',
        'Deli / Bodega', 'Mexican Restaurant', 'American Restaurant', 'BBQ Joint',
        'Fast Food Restaurant', 'Bar', 'Cupcake Shop', 'Candy Store', 'Pizza Place',
        'Sandwich Place', 'German Restaurant', 'Latin American Restaurant', 'Café',
        'Breakfast Spot', 'Malaysian Restaurant', 'Diner', 'Bakery', 'Fried Chicken Joint',
        'Snack Place', 'Seafood Restaurant', 'Salad Place', 'Wings Joint', 'Japanese Restaurant',
        'Falafel Restaurant', 'Middle Eastern Restaurant', 'Asian Restaurant', 'Beer Garden',
        'Ramen /  Noodle House', 'Hot Dog Joint', 'Cajun / Creole Restaurant', 'Mac & Cheese Joint',
        'Korean Restaurant', 'Sushi Restaurant', 'Gastropub', 'Caribbean Restaurant', 
        'African Restaurant', 'Cuban Restaurant', 'Indian Restaurant', 'Dessert Shop',
        'Thai Restaurant', 'Soup Place', 'Taco Place', 'Steakhouse', 'Dumpling Restaurant',
        'Vietnamese Restaurant', 'Southern / Soul Food Restaurant', 'Tapas Restaurant',
        'Filipino Restaurant', 'Brazilian Restaurant', 'Australian Restaurant', 
        'Eastern European Restaurant', 'Swiss Restaurant', 'Dim Sum Restaurant',
        'Mobile Phone Shop', 'Automotive Shop', 'Clothing Store', 'Electronics Store', 
        'Tattoo Parlor', 'Department Store', 'Hardware Store', 'Bookstore', 'Toy / Game Store',
        'Miscellaneous Shop', 'Furniture / Home Store', 'Bridal Shop', 'Paper / Office Supplies Store',
        'Convenience Store', 'Hobby Shop', 'Pet Store', 'Jewelry Store', 'Camera Store', 
        'Thrift / Vintage Store', 'Antique Shop', 'Market', 'Flea Market', 'Garden Center',
        'Salon / Barbershop', 'Cosmetics Shop', 'Bank', 'Financial or Legal Service',
        'Professional & Other Places', 'Design Studio', 'Laundry Service', 'Smoke Shop',
        'Post Office', 'Tattoo Parlor', 'Tanning Salon', 'Government Building', 'Office',
        'Other Nightlife', 'Building', 'Spanish Restaurant', 'Factory', 'Burrito Place',
        'Chinese Restaurant', 'Bagel Shop', 'Vegetarian / Vegan Restaurant', 'Donut Shop',
        'Sporting Goods Shop', 'French Restaurant', 'Italian Restaurant', 'Food Truck', 'Restaurant',
        'Tea Room', 'Brewery', 'Recycling Facility', 'Mediterranean Restaurant', 'Gift Shop', 'Food',
        'South American Restaurant', 'Molecular Gastronomy Restaurant', 'Scandinavian Restaurant',
        'Military Base', 'City'
    ],
    'Educational': [
        'Student Center', 'University', 'College Academic Building', 'Community College',
        'General College & University', 'College & University', 'Library', 'Law School', 
        'Trade School', 'Nursery School', 'Elementary School', 'Middle School', 
        'High School', 'College Stadium', 'School'
    ],
    'Transportation': [
        'Subway', 'Bus Station', 'Light Rail', 'Airport', 'Train Station', 'Parking', 
        'General Travel', 'Rental Car Location', 'Taxi', 'Ferry', 'Road', 'Harbor / Marina',
        'Bridge', 'Gas Station / Garage', 'River', 'Travel', 'Travel & Transport', 'Moving Target'
    ],
    'Culture & Leisure': [
        'Arts & Crafts Store', 'Music Venue', 'Movie Theater', 'Scenic Lookout', 'Theater', 
        'General Entertainment', 'Bowling Alley', 'Arcade', 'Comedy Club', 'Museum', 
        'Performing Arts Venue', 'Event Space', 'Art Museum', 'Concert Hall', 'Zoo', 
        'Aquarium', 'Casino', 'Science Museum', 'Racetrack', 'Fair', 'Music Store',
        'Stadium', 'Art Gallery', 'Park', 'Campground', 'Other Great Outdoors',
        'Beach', 'Playground', 'Pool Hall', 'Plaza', 'Outdoors & Recreation', 
        'Sculpture Garden', 'Garden', 'Travel Lounge', 'Rest Area', 'Convention Center',
        'Historic Site', 'Mall', 'Synagogue', 'Church', 'Cemetery', 'Temple', 'Shrine',
        'Arts & Entertainment', 'Spiritual Center'
    ],
    'Healthcare & Welfare': [
        'Gym / Fitness Center', 'Medical Center', 'Drugstore / Pharmacy', 'Spa / Massage',
        'Athletic & Sport', 'Pool', 'Animal Shelter', 'Funeral Home'
    ]
}

# 매핑 함수 
def map_to_final_category(category_name):
    for key, values in final_category_mapping.items():
        if category_name in values:
            return key
    return 'Else'

# trajectory_id 생성 
def make_trajectory_id(df):
    df['LocalTime'] = pd.to_datetime(df['LocalTime'])
    df['date'] = df['LocalTime'].dt.date
    df['TrajectoryId'] = df['UserId'].astype(str) + '_' + df['date'].astype(str)
    df.drop(columns=['date'], inplace=True)
    df = df.sort_values(by=['TrajectoryId', 'LocalTime'])
    return df

In [11]:
# 열이름 일치
nyc_train = make_same_column_names(nyc_train)
nyc_val = make_same_column_names(nyc_val)
nyc_test = make_same_column_names(nyc_test)

# 공휴일 정보 추가
nyc_train = add_holidays_nyc(nyc_train)
nyc_val = add_holidays_nyc(nyc_val)
nyc_test = add_holidays_nyc(nyc_test)

# 상위 카테고리 추가
nyc_train['UpperCategory'] = nyc_train['PoiCategoryName'].apply(map_to_final_category)
nyc_val['UpperCategory'] = nyc_val['PoiCategoryName'].apply(map_to_final_category)
nyc_test['UpperCategory'] = nyc_test['PoiCategoryName'].apply(map_to_final_category)

/tmp/ipykernel_80052/3883936841.py:35: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns, UTC] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df['is_holiday'] = df['LocalTime'].isin(us_holidays)
/tmp/ipykernel_80052/3883936841.py:35: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns, UTC] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df['is_holiday'] = df['LocalTime'].isin(us_holidays)
/tmp/ipykernel_80052/3883936841.py:35: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns, UTC] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  

In [17]:
nyc_train_traj = make_trajectory_id(nyc_train)
nyc_train_traj.head()

print(len(nyc_train_traj['TrajectoryId'].unique()))

#nyc_train_traj['UserId'].unique()
traj_count = nyc_train_traj.groupby('TrajectoryId').size()
traj4over = traj_count[traj_count >= 4]  # 1개 이상인 trajectory만 선택
len(traj4over)

28443


6923

In [ ]:

# 저장
nyc_train.to_csv('../data/nyc/raw/NYC_train.csv', index=False)
nyc_val.to_csv('../data/nyc/raw/NYC_val.csv', index=False)
nyc_test.to_csv('../data/nyc/raw/NYC_test.csv', index=False)

# TKY 데이터

In [3]:
tky_df = FileReader.read_dataset(file_name='dataset_TSMC2014_TKY.txt', dataset_name='tky')

# 필터링 적용
tky_df = FileReader.do_filter(tky_df, poi_min_freq=10, user_min_freq=10)

In [4]:
tky_df

,UserId,PoiId,PoiCategoryId,PoiCategoryName,Latitude,Longitude,TimezoneOffset,UTCTime,UTCTimeOffset,UTCTimeOffsetEpoch,UTCTimeOffsetWeekday,UTCTimeOffsetHour,UTCTimeOffsetDay,UserRank
2,114,4c16fdda96040f477cc473a5,4d954b0ea243a5684a65b473,Convenience Store,35.714542,139.480065,540,2012-04-03 19:12:07,2012-04-04 04:12:07,1333480327,2,4,2012-04-04,1.0
7,114,4b3eae5cf964a520b4a025e3,4bf58dd8d48988d129951735,Train Station,35.700253,139.480255,540,2012-04-03 19:35:36,2012-04-04 04:35:36,1333481736,2,4,2012-04-04,2.0
8,1635,4cca7bd67965b60c80f0858a,4bf58dd8d48988d162941735,Other Great Outdoors,35.755759,139.733573,540,2012-04-03 19:51:50,2012-04-04 04:51:50,1333482710,2,4,2012-04-04,1.0
9,2033,4b5c7671f964a520083129e3,4bf58dd8d48988d1d1941735,Ramen / Noodle House,35.693121,139.699447,540,2012-04-03 19:51:59,2012-04-04 04:51:59,1333482719,2,4,2012-04-04,1.0
10,589,4b5ed39cf964a520079a29e3,4bf58dd8d48988d1eb931735,Airport,35.548963,139.784611,540,2012-04-03 19:59:06,2012-04-04 04:59:06,1333483146,2,4,2012-04-04,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573695,2277,4b82669cf964a5209ed130e3,4bf58dd8d48988d172941735,Post Office,35.710080,139.790999,540,2013-02-16 02:33:48,2013-02-16 11:33:48,1360982028,5,11,2013-02-16,275.0
573697,2277,4b56c4c5f964a520c41a28e3,4bf58dd8d48988d1fd931735,Subway,35.709845,139.790801,540,2013-02-16 02:34:32,2013-02-16 11:34:32,1360982072,5,11,2013-02-16,276.0
573698,326,4bab3456f964a5204d993ae3,4bf58dd8d48988d1e9931735,Music Venue,35.656939,139.702518,540,2013-02-16 02:34:35,2013-02-16 11:34:35,1360982075,5,11,2013-02-16,254.0
573699,853,4b559c09f964a520efe827e3,4bf58dd8d48988d129951735,Train Station,35.858740,139.657286,540,2013-02-16 02:34:53,2013-02-16 11:34:53,1360982093,5,11,2013-02-16,102.0


* TKY 추가 전처리

In [19]:
# 1. 열이름 일치
tky_df.rename(columns={'UTCTimeOffset': 'LocalTime'}, inplace=True)

# 2. 공휴일 정보 추가 및 시간대 확인
def add_holidays_tky(df):
    df['LocalTime'] = pd.to_datetime(df['LocalTime'])  # 또는 다른 시간 컬럼
    
    japan_holidays = [
        # 2012
        datetime(2012, 4, 29).date(), datetime(2012, 4, 30).date(),
        datetime(2012, 5, 3).date(), datetime(2012, 5, 4).date(), datetime(2012, 5, 5).date(),
        datetime(2012, 7, 16).date(), datetime(2012, 9, 17).date(), datetime(2012, 9, 22).date(),
        datetime(2012, 10, 8).date(), datetime(2012, 11, 3).date(), datetime(2012, 11, 23).date(),
        datetime(2012, 12, 23).date(), datetime(2012, 12, 24).date(),
        # 2013
        datetime(2013, 1, 1).date(), datetime(2013, 1, 14).date(), datetime(2013, 2, 11).date()
    ]
    
    df['date_only'] = df['LocalTime'].dt.date
    df['is_weekend'] = df['LocalTime'].dt.weekday.isin([5, 6])  # 토/일
    df['is_holiday'] = df['date_only'].isin(japan_holidays)
    
    # 주말 or 공휴일
    df['Holiday'] = df['is_weekend'] | df['is_holiday']
    
    # 하루 기준 상대 시간 추가
    df['NormInDayTime'] = (df['LocalTime'].dt.hour * 3600 + df['LocalTime'].dt.minute * 60 + df['LocalTime'].dt.second) / (24*3600)  # 하루를 1로 정규화
    
    # 정리
    df.drop(columns=['is_weekend', 'is_holiday', 'date_only'], inplace=True)
    
    return df


final_category_mapping_tky = {
    'Residential': [
        'Neighborhood', 'Home (private)', 'Residential Building (Apartment / Condo)',
        'Housing Development', 'Sorority House'
    ],
    'Commercial/Services': [
        'Convention Center', 'Japanese Restaurant', 'Electronics Store', 'Cafï¿½',
        'Fast Food Restaurant', 'Convenience Store', 'Paper / Office Supplies Store',
        'Chinese Restaurant', 'Office', 'Bookstore', 'Hobby Shop', 'Bar',
        'Miscellaneous Shop', 'Toy / Game Store', 'Ramen /  Noodle House', 'Smoke Shop',
        'Shrine', 'Plaza', 'Building', 'Italian Restaurant', 'General Entertainment',
        'Clothing Store', 'Hardware Store', 'Coffee Shop', 'Fried Chicken Joint',
        'Food & Drink Shop', 'Dessert Shop', 'Restaurant', 'Mall', 'Bakery',
        'Indian Restaurant', 'Post Office', 'Government Building',
        'Drugstore / Pharmacy', 'Diner', 'Soup Place', 'Burger Joint', 'Racetrack',
        'Department Store', 'Record Shop', 'Music Venue', 'General Travel',
        'Furniture / Home Store', 'Camera Store', 'Sushi Restaurant', 'Hotel',
        'Arts & Crafts Store', 'Bike Shop', 'Mobile Phone Shop', 'Recycling Facility',
        'Antique Shop', 'Donut Shop', 'Deli / Bodega', 'Ice Cream Shop', 'Asian Restaurant',
        'Steakhouse', 'Video Store', 'Video Game Store', 'Dumpling Restaurant',
        'Sandwich Place', 'Internet Cafe', 'Military Base', 'Sporting Goods Shop',
        'Bank', 'Music Store', 'Travel Lounge', 'Seafood Restaurant',
        'Travel & Transport', 'Breakfast Spot', 'Gift Shop', 'Athletic & Sport',
        'Pizza Place', 'BBQ Joint', 'Gaming Cafe', 'Salon / Barbershop', 'Hot Dog Joint',
        'American Restaurant', 'Brewery', 'Harbor / Marina', 'Middle Eastern Restaurant',
        'Automotive Shop', 'Fish & Chips Shop', 'Comedy Club', 'Gastropub',
        'Scenic Lookout', 'Caribbean Restaurant', 'Shop & Service', 'French Restaurant',
        'Thai Restaurant', 'Brazilian Restaurant', 'Moving Target', 'Laundry Service',
        'Flower Shop', 'River', 'Spiritual Center', 'Playground', 'Mexican Restaurant',
        'Car Dealership', 'Candy Store', 'Food', 'Motorcycle Shop', 'Wings Joint',
        'Tea Room', 'Board Shop', 'Mediterranean Restaurant', 'Tanning Salon',
        'Food Truck', 'Thrift / Vintage Store', 'Pool', 'Embassy / Consulate',
        'Snack Place', 'Professional & Other Places', 'Korean Restaurant',
        'Cosmetics Shop', 'Factory', 'Pet Store', 'Bike Rental / Bike Share'
    ],
    'Educational': [
        'University', 'College Academic Building', 'General College & University',
        'Student Center', 'School', 'High School', 'Community College', 'Trade School',
        'Elementary School', 'Medical School', 'College & University', 'Nursery School',
        'College Stadium'
    ],
    'Transportation': [
        'Train Station', 'Subway', 'Bus Station', 'Road', 'Light Rail',
        'Gas Station / Garage', 'Rest Area', 'Parking', 'Airport', 'Ferry', 'Taxi',
        'Bridge'
    ],
    'Culture & Leisure': [
        'Event Space', 'Stadium', 'Arcade', 'Temple', 'Gym / Fitness Center', 'Park',
        'Other Great Outdoors', 'Spa / Massage', 'Movie Theater', 'Sculpture Garden',
        'Aquarium', 'Zoo', 'Art Museum', 'Performing Arts Venue', 'Library',
        'Science Museum', 'Church', 'Historic Site', 'History Museum', 'Bowling Alley',
        'Garden', 'Concert Hall', 'Casino', 'Other Nightlife', 'Art Gallery',
        'Beer Garden', 'Theater', 'Museum', 'Beach', 'Public Art', 'Garden Center',
        'Outdoors & Recreation', 'Nightlife Spot', 'Cemetery'
    ],
    'Healthcare & Welfare': [
        'Medical Center'
    ]
}

def map_to_category(category_name):
    for key, values in final_category_mapping_tky.items():
        if category_name in values:
            return key
    return 'Else'

In [20]:
# 1. 열 이름 일치
tky_df = make_same_column_names(tky_df)

# 2. 공휴일 정보 추가 및 시간대 확인
tky_df = add_holidays_tky(tky_df)

# 3. 상위 카테고리 추가
tky_df['UpperCategory'] = tky_df['PoiCategoryName'].apply(map_to_category)

In [21]:
# 데이터 분할 및 저장
tky_df = FileReader.split_train_test(tky_df)

train_tky = tky_df[tky_df['SplitTag'] == 'train'] 
val_tky = tky_df[tky_df['SplitTag'] == 'validation']
test_tky = tky_df[tky_df['SplitTag'] == 'test']

In [26]:
tky_train_traj = make_trajectory_id(train_tky)
tky_train_traj.head()

print(len(tky_train_traj['TrajectoryId'].unique()))

tky_train_traj['UserId'].unique()
traj_count = tky_train_traj.groupby('TrajectoryId').size()
traj4over = traj_count[traj_count >= 5]  # 1개 이상인 trajectory만 선택
len(traj4over)

/tmp/ipykernel_80052/3883936841.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LocalTime'] = pd.to_datetime(df['LocalTime'])
/tmp/ipykernel_80052/3883936841.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['LocalTime'].dt.date
/tmp/ipykernel_80052/3883936841.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

96276


20333

In [ ]:
# 저장
train_tky.to_csv('../data/tky/raw/TKY_train.csv', index=False)
val_tky.to_csv('../data/tky/raw/TKY_val.csv', index=False)
test_tky.to_csv('../data/tky/raw/TKY_test.csv', index=False)

# CA 데이터

In [10]:
ca_df = FileReader.read_dataset(file_name='dataset_gowalla_ca_ne.csv', dataset_name='ca')
ca_df

,UserId,PoiId,PoiCategoryId,Latitude,Longitude,UTCTime,UTCTimeOffset,PoiCategoryName,UTCTimeOffsetEpoch,UTCTimeOffsetWeekday,UTCTimeOffsetHour,UTCTimeOffsetDay,UserRank
0,0,19542,45,37.616356,-122.386150,2010-08-19T21:59:09Z,2010-08-19 21:59:09,Airport,1282222749,3,21,2010-08-19,20.0
1,0,19542,45,37.616356,-122.386150,2010-06-24T14:27:35Z,2010-06-24 14:27:35,Airport,1277357255,3,14,2010-06-24,13.0
2,0,19542,45,37.616356,-122.386150,2010-06-06T18:48:32Z,2010-06-06 18:48:32,Airport,1275817712,6,18,2010-06-06,1.0
3,4,19542,45,37.616356,-122.386150,2010-06-19T15:37:36Z,2010-06-19 15:37:36,Airport,1276929456,5,15,2010-06-19,73.0
4,8,19542,45,37.616356,-122.386150,2010-10-07T03:20:57Z,2010-10-07 03:20:57,Airport,1286389257,3,3,2010-10-07,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
636507,196561,2657838,457,37.526711,-122.000513,2010-09-21T22:38:47Z,2010-09-21 22:38:47,Burger King,1285076327,1,22,2010-09-21,37.0
636508,196561,4062193,160,37.522527,-122.005202,2010-09-19T17:57:42Z,2010-09-19 17:57:42,Indian,1284886662,6,17,2010-09-19,31.0
636509,196561,3793703,38,37.530799,-122.015335,2010-09-16T00:17:48Z,2010-09-16 00:17:48,City Park,1284563868,3,0,2010-09-16,8.0
636510,196561,1109654,15,37.523095,-122.004770,2010-09-15T21:39:42Z,2010-09-15 21:39:42,Mexican,1284554382,2,21,2010-09-15,5.0


In [11]:
ca_df = FileReader.do_filter(ca_df, poi_min_freq=10, user_min_freq=10)

# 1. 열이름 일치
ca_df.rename(columns={'UTCTimeOffset': 'LocalTime'}, inplace=True)

ca_df = FileReader.split_train_test(ca_df)

train_ca = ca_df[ca_df['SplitTag'] == 'train'] 
val_ca = ca_df[ca_df['SplitTag'] == 'validation']
test_ca = ca_df[ca_df['SplitTag'] == 'test']

train_ca.to_csv('../data/ca/raw/CA_train.csv', index=False)
val_ca.to_csv('../data/ca/raw/CA_val.csv', index=False)
test_ca.to_csv('../data/ca/raw/CA_test.csv', index=False)